# **Getting Started Code For [Data Sprint #84](https://dphi.tech/challenges/datathon/) on DPhi**

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [1]:
!wget https://s3.us-west-1.wasabisys.com/dphi/public-datasets/Data%20Sprint%2084%20Caddy%20Gesture%20Recognition/Caddy_gestures.zip

--2022-07-17 15:13:29--  https://s3.us-west-1.wasabisys.com/dphi/public-datasets/Data%20Sprint%2084%20Caddy%20Gesture%20Recognition/Caddy_gestures.zip
Resolving s3.us-west-1.wasabisys.com (s3.us-west-1.wasabisys.com)... 38.146.40.53, 38.146.40.51
Connecting to s3.us-west-1.wasabisys.com (s3.us-west-1.wasabisys.com)|38.146.40.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523114835 (1.4G) [application/x-zip-compressed]
Saving to: ‘Caddy_gestures.zip’

Caddy_gestures.zip  100%[===================>]   1.42G  38.3MB/s    in 43s     

2022-07-17 15:14:14 (33.4 MB/s) - ‘Caddy_gestures.zip’ saved [1523114835/1523114835]



In [2]:
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
!7za x /content/Caddy_gestures.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 1523114835 bytes (1453 MiB)

Extracting archive: /content/Caddy_gestures.zip
 22% 4096 Open              --
Path = /content/Caddy_gestures.zip
Type = zip
Physical Size = 1523114835

  0%      1% 296 - test/Image_1265.jpg                                2% 549 - test/Image_1493.jpg                                4% 792 - test/Image_1711.jpg                                5% 961 - test/Image_1864.jpg                              

In [4]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set

## Loading and preparing training data
The train and test images are given in two different folders - 'train' and 'test'. The labels of train images are given in a csv file 'Train.csv' with respective image id (i.e. image file name).

#### Getting the labels of the images

In [5]:
labels = pd.read_csv("/content/Training_set.csv")   # loading the labels
labels.head()           # will display the first five rows in labels dataframe

,filename,label
0,Image_1.jpg,genova-A
1,Image_2.jpg,brodarski-B
2,Image_3.jpg,genova-A
3,Image_4.jpg,genova-A
4,Image_5.jpg,biograd-A


In [6]:
labels['label'].unique()

array(['genova-A', 'brodarski-B', 'biograd-A', 'biograd-C', 'brodarski-C',
       'biograd-B', 'brodarski-A', 'brodarski-D'], dtype=object)

In [7]:
labels.tail()            # will display the last five rows in labels dataframe

,filename,label
12929,Image_12930.jpg,biograd-A
12930,Image_12931.jpg,brodarski-D
12931,Image_12932.jpg,genova-A
12932,Image_12933.jpg,biograd-C
12933,Image_12934.jpg,biograd-C


#### Getting images file path

In [8]:
file_paths = [[fname, '/content/train/' + fname] for fname in labels['filename']]

#### Confirming if no. of labels is equal to no. of images

In [9]:
# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
    print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

Number of labels i.e.  12934 matches the number of filenames i.e.  12934


#### Converting the file_paths to dataframe

In [10]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,/content/train/Image_1.jpg
1,Image_2.jpg,/content/train/Image_2.jpg
2,Image_3.jpg,/content/train/Image_3.jpg
3,Image_4.jpg,/content/train/Image_4.jpg
4,Image_5.jpg,/content/train/Image_5.jpg


#### Combining the labels with the images

In [11]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()       

,filename,filepaths,label
0,Image_1.jpg,/content/train/Image_1.jpg,genova-A
1,Image_2.jpg,/content/train/Image_2.jpg,brodarski-B
2,Image_3.jpg,/content/train/Image_3.jpg,genova-A
3,Image_4.jpg,/content/train/Image_4.jpg,genova-A
4,Image_5.jpg,/content/train/Image_5.jpg,biograd-A


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])

The 'train_data' dataframe contains all the image id, their locations and their respective labels. Now the training data is ready.

## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [13]:
data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size),cv2.INTER_AREA)      # resizing the image array
  data.append([new_img_array, train_data['label'][i]])

In [14]:
# image pixels of a image
data[5]

[array([[ 99, 102, 101, ..., 132, 128, 124],
        [102, 101, 104, ..., 131, 129, 126],
        [100, 104, 105, ..., 131, 132, 128],
        ...,
        [ 61,  61,  63, ...,  73,  71,  71],
        [ 60,  61,  61, ...,  71,  69,  69],
        [ 59,  61,  62, ...,  70,  69,  69]], dtype=uint8), 2]

#### Shuffle the data

In [15]:
np.random.shuffle(data)

#### Separating the images and labels


In [16]:
x = []
y = []
for image in data:
  x.append(image[0])
  y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [17]:
np.unique(y, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([2419, 1365, 2527,  318,  122, 1082,  453, 4648]))

#### Splitting the data into Train and Validation Set
We want to check the performance of the model that we built. For this purpose, we always split (both independent and dependent data) the given data into training set which will be used to train the model, and test set which will be used to check how accurately the model is predicting outcomes.

For this purpose we have a class called 'train_test_split' in the 'sklearn.model_selection' module.

In [18]:
x =  x.reshape(-1, 100, 100, 1)

In [19]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2, random_state = 42)

## Building Model
Now we are finally ready, and we can train the model.

There are many machine learning or deep learning models like Random Forest, Decision Tree, Multi-Layer Perceptron (MLP), Convolution Neural Network (CNN), etc. to say you some.


Then we would feed the model both with the data (X_train) and the answers for that data (y_train)

In [20]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')
])

In [21]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
cnn.fit(X_train, y_train, epochs=10, batch_size=4)

Epoch 1/10
2587/2587 [==============================] - 24s 5ms/step - loss: 0.3863 - accuracy: 0.8880
Epoch 2/10
2587/2587 [==============================] - 11s 4ms/step - loss: 0.1185 - accuracy: 0.9659
Epoch 3/10
2587/2587 [==============================] - 11s 4ms/step - loss: 0.0860 - accuracy: 0.9762
Epoch 4/10
2587/2587 [==============================] - 11s 4ms/step - loss: 0.0468 - accuracy: 0.9887
Epoch 5/10
2587/2587 [==============================] - 11s 4ms/step - loss: 0.0413 - accuracy: 0.9905
Epoch 6/10
2587/2587 [==============================] - 11s 4ms/step - loss: 0.0474 - accuracy: 0.9919
Epoch 7/10
2587/2587 [==============================] - 11s 4ms/step - loss: 0.0532 - accuracy: 0.9891
Epoch 8/10
2587/2587 [==============================] - 11s 4ms/step - loss: 0.0466 - accuracy: 0.9898
Epoch 9/10
2587/2587 [==============================] - 11s 4ms/step - loss: 0.0298 - accuracy: 0.9944
Epoch 10/10
2587/2587 [==============================] - 11s 4ms/step - l

## Validate the model
Wonder🤔 how well your model learned! Lets check its performance on the X_val data.

In [23]:
cnn.evaluate(X_val, y_val)

81/81 [==============================] - 1s 7ms/step - loss: 0.0569 - accuracy: 0.9927


[0.056928835809230804, 0.9926555752754211]

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

In [24]:
# Loading the order of the image's name that has been provided
test_image_order = pd.read_csv("/content/Testing_set.csv")
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


#### Getting images file path

In [25]:
file_paths = [[fname, '/content/test/' + fname] for fname in test_image_order['filename']]

#### Confirm if number of images in test folder is same as number of image names in 'Testing_set_face_mask.csv'

In [26]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
    print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
    print('Number of image names does not match the number of filepaths')

Number of image names i.e.  5544 matches the number of file paths i.e.  5544


#### Converting the file_paths to dataframe

In [27]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.tail()

,filename,filepaths
5539,Image_5540.jpg,/content/test/Image_5540.jpg
5540,Image_5541.jpg,/content/test/Image_5541.jpg
5541,Image_5542.jpg,/content/test/Image_5542.jpg
5542,Image_5543.jpg,/content/test/Image_5543.jpg
5543,Image_5544.jpg,/content/test/Image_5544.jpg


## Data Pre-processing on test_data


In [28]:
test_pixel_data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
missing_files = []
for i in range(len(test_images)):
    img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

    new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array

    test_pixel_data.append(new_img_array)


In [29]:
test_pixel_data = np.array(test_pixel_data)

In [30]:
test_pixel_data =  test_pixel_data.reshape(-1, 100, 100, 1)

### Make Prediction on Test Dataset
Time to make a submission!!!

In [31]:
pred = cnn.predict(test_pixel_data)

In [32]:
# The predicted values are the probabilities value
pred[0]

array([4.4330065e-16, 0.0000000e+00, 8.4743508e-13, 0.0000000e+00,
       8.3414466e-34, 1.6112226e-38, 0.0000000e+00, 1.0000000e+00],
      dtype=float32)

The above values are probability values. We need to convert it into respective classes. We can use np.argmax for the same.

In [33]:
prediction = []
for value in pred:
  prediction.append(np.argmax(value))

In [34]:
predictions = le.inverse_transform(prediction)

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [35]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [36]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Well Done! 👍**
You are all set to make a submission. Let's head to the **[challenge page](https://dphi.tech/challenges/data-sprint-41/142/submit)** to make the submission.